In [2]:
filename = r"C:\Users\Tanner\Google Drive\Documents\Work\Papers\Morgan - Article - 2020 - Development of a framework for sequential Bayesian design of experiments Application to a pilot scale solvent based CO2 capture process.pdf"

import tabula
import pandas as pd

# Extract tables from PDF into a list of DataFrames
tables = tabula.read_pdf(filename, pages='18', multiple_tables=True)

# Now you can work with the first table:
df = tables[0]
print(df.head())

JavaNotFoundError: `java` command is not found from this Python process.Please ensure Java is installed and PATH is set for `java`